# Ejercicio 3 - guía 9

Un hospital está analizando los tiempos de respuesta (en minutos) desde que se recibe una
llamada de emergencia hasta la llegada de la ambulancia, expresados como desviaciones respecto
al objetivo de 10 minutos (por ejemplo, -1 significa 9 minutos y +1 significa 11 minutos).

## a)
Utiliza dos registros iniciales: y = [0,2]. Asume que el tiempo medio de respuesta sigue una
distribución normal con media de 0 minutos y desviación estándar de 1000 minutos, y que la
variabilidad sigue una distribución exponencial con parámetro lambda de 0.0001. Construye
un modelo bayesiano para estimar el tiempo medio y la variabilidad.

### Desarrollo

Armo el modelo:

In [3]:
import pymc as pm

datos_observados = [0, 2]

with pm.Model() as modelo:
    # Priors
    mu = pm.Normal('mu', mu= 0, sigma= 1000)
    sigma = pm.Exponential('sigma', 0.0001)
    
    # Likelihood
    tiempo_medio_respuesta = pm.Normal('tiempo_medio_respuesta', mu=mu, sigma=sigma, observed=datos_observados)
    
    # Sampleo la posterior
    idata = pm.sample(10000, tune=1000, cores=1)


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [mu, sigma]


c:\Users\gerar\OneDrive\Documentos\Mis Archivos\Estadistica e 
Inferencia\estat_inferencia\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 2 chains for 1_000 tune and 10_000 draw iterations (2_000 + 20_000 draws total) took 48 seconds.
There were 7752 divergences after tuning. Increase `target_accept` or reparameterize.
We recommend running at least 4 chains for robust computation of convergence diagnostics
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Observo su desempeño:

In [4]:
pm.summary(idata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu,0.686,343.092,-696.290,728.127,5.757,10.967,3105.0,1688.0,1.41
sigma,549.415,1447.431,2.936,2086.154,177.757,126.250,4.0,21.0,1.46


## b)
Añade más observaciones: y = [0, 2, -1, 1, -0.5, 0.8, 0.3, -0.2, 1.2, -0.7]. Analiza los cambios en
las estimaciones

### Desarrollo

Armo el mismo modelo:

In [5]:
datos_observados_new = [0, 2, -1, 1, -0.5, 0.8, 0.3, -0.2, 1.2, -0.7]

with pm.Model() as modelo:
    # Priors
    mu = pm.Normal('mu', mu= 0, sigma= 1000)
    sigma = pm.Exponential('sigma', 0.0001)
    
    # Likelihood
    tiempo_medio_respuesta = pm.Normal('tiempo_medio_respuesta', mu=mu, sigma=sigma, observed=datos_observados_new)
    
    # Sampleo la posterior
    idata_better = pm.sample(10000, tune=1000, cores=1)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [mu, sigma]


c:\Users\gerar\OneDrive\Documentos\Mis Archivos\Estadistica e 
Inferencia\estat_inferencia\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 2 chains for 1_000 tune and 10_000 draw iterations (2_000 + 20_000 draws total) took 56 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


Y ahora observo su desempeño, en teoría mejorado:

In [6]:
pm.summary(idata_better)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu,0.284,0.369,-0.407,0.959,0.004,0.004,12330.0,9892.0,1.0
sigma,1.117,0.346,0.625,1.715,0.004,0.003,10141.0,9992.0,1.0


## c)
Proponé mejoras al modelo inicial considerando tiempos de respuesta típicos en emergencias.

### Desarrollo

Armo un modelo nuevo cambiando un poco los priors:

* Considero a la media centrada igualmente en 0 pero con menor desviación, **una de 10**

* Considero a la desviación igualmente como una exponencial pero con **lambda igual a 2**, 
  lo que logra que el tiempo de respuesta sea "más rápido" ya que la desviación es más pequeña de algún modo

Si bien las nuevas elecciones de priors están pensados para tiempos de respuesta en emergencias, la elección de algunos parámetros
es arbitraria.

In [17]:
datos_observados_new = [0, 2, -1, 1, -0.5, 0.8, 0.3, -0.2, 1.2, -0.7]

with pm.Model() as modelo:
    # Priors
    mu = pm.Normal('mu', mu= 0, sigma= 10)
    sigma = pm.Exponential('sigma', 2)

    # Likelihood
    tiempo_medio_respuesta = pm.Normal('tiempo_medio_respuesta', mu=mu, sigma=sigma, observed=datos_observados_new)
    
    # Sampleo la posterior
    idata_better = pm.sample(10000, tune=1000, cores=1)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [mu, sigma]


c:\Users\gerar\OneDrive\Documentos\Mis Archivos\Estadistica e 
Inferencia\estat_inferencia\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 2 chains for 1_000 tune and 10_000 draw iterations (2_000 + 20_000 draws total) took 56 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


Y ahora evaluo al nuevo modelo:

In [18]:
pm.summary(idata_better)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu,0.290,0.314,-0.298,0.886,0.003,0.002,14389.0,11292.0,1.0
sigma,0.966,0.225,0.590,1.381,0.002,0.002,12036.0,11253.0,1.0
